![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/fastai/train-with-custom-docker/fastai-with-custom-docker.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train a model using a custom Docker image

In this tutorial, learn how to use a custom Docker image when training models with Azure Machine Learning.

The example scripts in this article are used to classify pet images by creating a convolutional neural network. 

## Set up the experiment
This section sets up the training experiment by initializing a workspace, creating an experiment, and uploading the training data and training scripts.

### Initialize a workspace
The Azure Machine Learning workspace is the top-level resource for the service. It provides you with a centralized place to work with all the artifacts you create. In the Python SDK, you can access the workspace artifacts by creating a `workspace` object.

Create a workspace object from the config.json file.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

### Prepare scripts
Create a directory titled `fastai-example`.

In [ ]:
import os
os.makedirs('fastai-example', exist_ok=True)

Then run the cell below to create the training script train.py in the directory.

In [ ]:
%%writefile fastai-example/train.py

from fastai.vision.all import *

path = untar_data(URLs.PETS)
path.ls()

files = get_image_files(path/"images")
len(files)

#(Path('/home/ashwin/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_102.jpg'),Path('/home/ashwin/.fastai/data/oxford-iiit-pet/images/great_pyrenees_102.jpg'))

def label_func(f): return f[0].isupper()

#To get our data ready for a model, we need to put it in a DataLoaders object. Here we have a function that labels using the file names, so we will use ImageDataLoaders.from_name_func. There are other factory methods of ImageDataLoaders that could be more suitable for your problem, so make sure to check them all in vision.data.

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

#We have passed to this function the directory we're working in, the files we grabbed, our label_func and one last piece as item_tfms: this is a Transform applied on all items of our dataset that will resize each imge to 224 by 224, by using a random crop on the largest dimension to make it a square, then resizing to 224 by 224. If we didn't pass this, we would get an error later as it would be impossible to batch the items together.

dls.show_batch()

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

### Define your environment
Create an environment object and enable Docker.

In [ ]:
from azureml.core import Environment

fastai_env = Environment("fastai")
fastai_env.docker.enabled = True

This specified base image supports the fast.ai library which allows for distributed deep learning capabilities. For more information, see the [fast.ai DockerHub](https://hub.docker.com/u/fastdotai). 

When you are using your custom Docker image, you might already have your Python environment properly set up. In that case, set the `user_managed_dependencies` flag to True in order to leverage your custom image's built-in python environment.

In [ ]:
fastai_env.docker.base_image = "fastdotai/fastai:latest"
fastai_env.python.user_managed_dependencies = True

To use an image from a private container registry that is not in your workspace, you must use `docker.base_image_registry` to specify the address of the repository as well as a username and password.

```python
fastai_env.docker.base_image_registry.address = "myregistry.azurecr.io"
fastai_env.docker.base_image_registry.username = "username"
fastai_env.docker.base_image_registry.password = "password"
```

It is also possible to use a custom Dockerfile. Use this approach if you need to install non-Python packages as dependencies and remember to set the base image to None. 

Specify docker steps as a string:
```python 
dockerfile = r""" \
FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04
RUN echo "Hello from custom container!" \
"""
```
Set base image to None, because the image is defined by dockerfile:
```python
fastai_env.docker.base_image = None \
fastai_env.docker.base_dockerfile = dockerfile
```
Alternatively, load the string from a file:
```python
fastai_env.docker.base_image = None \
fastai_env.docker.base_dockerfile = "./Dockerfile"
```

### Create or attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute
print(compute_target.get_status().serialize())

### Create a ScriptRunConfig
This ScriptRunConfig will configure your job for execution on the desired compute target.

In [ ]:
from azureml.core import ScriptRunConfig

fastai_config = ScriptRunConfig(source_directory='fastai-example',
                                script='train.py',
                                compute_target=compute_target,
                                environment=fastai_env)

### Submit your run
When a training run is submitted using a ScriptRunConfig object, the submit method returns an object of type ScriptRun. The returned ScriptRun object gives you programmatic access to information about the training run. 

In [ ]:
from azureml.core import Experiment

run = Experiment(ws,'fastai-custom-image').submit(fastai_config)
run.wait_for_completion(show_output=True)